### Data Analysis and Cleaning

In [ ]:
import numpy as np
import pandas as pd

#### Initial filtering with IMDB databases (now not in data folder, because they were too heavy)

In [ ]:
all_titles = pd.read_csv('data/title.basics.tsv', sep='\t')
all_ratings = pd.read_csv('data/title.ratings.tsv', sep='\t')

# Filter titles to only include movies and TV series
movies_and_series = all_titles[all_titles['titleType'].isin(['movie', 'tvSeries'])]

# Convert columns to correct data types
all_ratings['averageRating'] = pd.to_numeric(all_ratings['averageRating'], errors='coerce')
all_ratings['numVotes'] = pd.to_numeric(all_ratings['numVotes'], errors='coerce')

# Merge titles and ratings, keeping only movies and TV series
merged_data = movies_and_series.merge(all_ratings, on='tconst', how='inner')

# Filter movies/series with at least 1000 votes
df_filtered = merged_data[merged_data['numVotes'] >= 1000]

# Sort by rating (descending), then by numVotes (descending for tie-breaking)
df_sorted = df_filtered.sort_values(by=['averageRating', 'numVotes'], ascending=[False, False])

# Save to new TSV file
df_sorted.to_csv('data/filtered_sorted_with_ratings.tsv', sep='\t', index=False)

In [ ]:
# IMDb-style weighted rating system for top 10,000 movies and series

# Calculate overall statistics for the weighted rating formula
overall_mean_rating = df_filtered['averageRating'].mean()
min_votes_required = df_filtered['numVotes'].quantile(0.75)  # Use 75th percentile as minimum

print(f"Overall mean rating: {overall_mean_rating:.2f}")
print(f"Minimum votes threshold (75th percentile): {min_votes_required:.0f}")

# Apply Bayesian weighted rating formula
# Weighted Rating = (v / (v + m)) * R + (m / (v + m)) * C
# Where: v = votes, m = min votes, R = average rating, C = overall mean
def calculate_weighted_rating(row):
    v = row['numVotes']
    R = row['averageRating']
    m = min_votes_required
    C = overall_mean_rating
    
    weighted_rating = (v / (v + m)) * R + (m / (v + m)) * C
    return weighted_rating

# Add weighted rating column
df_filtered['weightedRating'] = df_filtered.apply(calculate_weighted_rating, axis=1)

# Sort by weighted rating (descending), then by numVotes (descending for tie-breaking)
df_weighted_sorted = df_filtered.sort_values(by=['weightedRating', 'numVotes'], ascending=[False, False])

# Get top 10,000 based on weighted ratings
top_10000_weighted = df_weighted_sorted.head(10000)

# Save the weighted top 10,000 to file
top_10000_weighted.to_csv('data/top_10000_weighted_ratings.tsv', sep='\t', index=False)

print(f"\nTop 10 movies/series by weighted rating:")
print(top_10000_weighted[['primaryTitle', 'averageRating', 'numVotes', 'weightedRating']].head(10))

Overall mean rating: 6.39
Minimum votes threshold (75th percentile): 10202

Top 10 movies/series by weighted rating:
                      primaryTitle  averageRating  numVotes  weightedRating
175388                Breaking Bad            9.5   2405005        9.486874
67193     The Shawshank Redemption            9.3   3104334        9.290476
153336  Avatar: The Last Airbender            9.3    417720        9.230683
129412                    The Wire            9.3    413512        9.229994
176712             Game of Thrones            9.2   2485498        9.188523
39197                The Godfather            9.2   2163561        9.186824
80066                 The Sopranos            9.2    545115        9.148422
162488             The Dark Knight            9.1   3079402        9.091060
308327             Attack on Titan            9.1    649760        9.058146
234194                   Aspirants            9.1    316623        9.015484

Top 10 movies/series by weighted rating:
     

C:\Users\franc\AppData\Local\Temp\ipykernel_10268\2729732123.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['weightedRating'] = df_filtered.apply(calculate_weighted_rating, axis=1)


### Web Scraping / API Calls to get Cast and Plot Description